In [1]:
import os
import cv2
import numpy as np
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers
from tensorflow.keras.models import load_model
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

In [2]:
train_dir = "../data/final/train"
test_dir  = "../data/final/test"

img_size = (224, 224)
batch_size = 16

from tensorflow.keras.applications.efficientnet import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    brightness_range=[0.8, 1.2],
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse'  # assuming you're using sparse categorical crossentropy
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse'
)


Found 9252 images belonging to 257 classes.
Found 514 images belonging to 257 classes.


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
NUM_CLASSES = len(os.listdir(train_dir))

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers[:-16]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = Dropout(0.3)(x)
predictions = Dense(NUM_CLASSES, activation='softmax', 
                    kernel_regularizer=regularizers.l2(0.001))(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', patience=3, factor=0.5, min_lr=1e-7)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,                 
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    filepath='best_fingerprint_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=100,
    callbacks=[lr_scheduler, early_stopping]
)


Epoch 1/100
579/579 [==============================] - 203s 334ms/step - loss: 7.1585 - accuracy: 0.0372 - val_loss: 6.3480 - val_accuracy: 0.0798 - lr: 1.0000e-04
Epoch 2/100
579/579 [==============================] - 139s 239ms/step - loss: 5.3183 - accuracy: 0.1876 - val_loss: 5.5021 - val_accuracy: 0.1459 - lr: 1.0000e-04
Epoch 3/100
579/579 [==============================] - 120s 207ms/step - loss: 4.2393 - accuracy: 0.3487 - val_loss: 5.0885 - val_accuracy: 0.2432 - lr: 1.0000e-04
Epoch 4/100
579/579 [==============================] - 122s 211ms/step - loss: 3.6529 - accuracy: 0.4631 - val_loss: 5.0401 - val_accuracy: 0.2490 - lr: 1.0000e-04
Epoch 5/100
579/579 [==============================] - 138s 239ms/step - loss: 3.2434 - accuracy: 0.5457 - val_loss: 5.1207 - val_accuracy: 0.2374 - lr: 1.0000e-04
Epoch 6/100
579/579 [==============================] - 116s 200ms/step - loss: 2.9766 - accuracy: 0.6091 - val_loss: 4.7660 - val_accuracy: 0.2879 - lr: 1.0000e-04
Epoch 7/100
579/

In [4]:
loss, accuracy = model.evaluate(test_generator)
print(f"\n Test Loss: {loss:.4f}")
print(f" Test Accuracy: {accuracy:.4f}")

33/33 [==============================] - 3s 67ms/step - loss: 4.2894 - accuracy: 0.3521

 Test Loss: 4.2894
 Test Accuracy: 0.3521


In [2]:
!pip install scikit-image


   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
    --------------------------------------- 0.3/12.9 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.9 MB 3.4 MB/s eta 0:00:04
   ---- ----------------------------------- 1.6/12.9 MB 3.6 MB/s eta 0:00:04
   ----- ---------------------------------- 1.8/12.9 MB 3.4 MB/s eta 0:00:04
   ----- ---------------------------------- 1.8/12.9 MB 3.4 MB/s eta 0:00:04
   ------ --------------------------------- 2.1/12.9 MB 1.8 MB/s eta 0:00:06
   --------- ------------------------------ 3.1/12.9 MB 2.3 MB/s eta 0:00:05
   ---------- ----------------------------- 3.4/12.9 MB 2.4 MB/s eta 0:00:04
   ---------- ----------------------------- 3.4/12.9 MB 2.4 MB/s eta 0:00:04
   ------------ --------------------------- 3.9/12.9 MB 2.0 MB/s eta 0:00:05
   -------------- ------------------------- 4.7/12.9 MB 2.2 MB/s eta 0:00:04
   ------------------ --------------------- 6.0/12.9 MB 2.6 MB/s eta 0:00:03
   ----------


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
